https://github.com/andrewgbruce/statistics-for-data-scientists/blob/master/src/chapter4.r

In [ ]:
house = read.csv('~/house_sales.csv',sep = '\t',header = TRUE)

In [ ]:
house

In [ ]:
house_model = lm(AdjSalePrice ~  SqFtTotLiving + PropertyType + SqFtFinBasement + SqFtLot  + NbrLivingUnits + Bathrooms + Bedrooms + BldgGrade + LandVal + YrBuilt + YrRenovated +  TrafficNoise + ImpsVal + NewConstruction, data = house, na.action=na.omit )

In [ ]:
summary(house_model)

In [ ]:
# Minimum p-values above, and Max t-values indicate best predictors.
# LAndVal and ImpsVal are so strong ,they're likely the only ones that will sirvive.  Maybe sqftLot and BldgGrade?

In [ ]:
#We can get a better idea of the best variables to include by doing several stepwise regressions to see which models improve accuracy. 
# The stepAIC() function doest this work for us.   

In [ ]:

library(MASS)


In [ ]:
step = stepAIC(house_model,direction="both")

In [ ]:
step

In [ ]:
#what if we stepAIC the result?
step2 = stepAIC(step,direction="both")

In [ ]:
step2

In [ ]:
#nothing.  Ok, then lets drop the low t scores/high p vals, and see what we get

In [ ]:
house_model2 = lm(AdjSalePrice ~  SqFtTotLiving + SqFtFinBasement + SqFtLot + NbrLivingUnits + Bedrooms + BldgGrade + LandVal  + YrRenovated +  ImpsVal + NewConstruction, data = house, na.action=na.omit )

In [ ]:
step2 = stepAIC(house_model2,direction="both")

In [ ]:
step

In [ ]:
step2

In [ ]:
install.packages('lubridate')
library(lubridate)

In [ ]:
sale_date = year(house$DocumentDate)

In [ ]:
# Weighted Varaibles, such as Sale Date

In [ ]:
house$wt = sale_date - 2005

In [ ]:
house

In [ ]:
house_model2 = lm(AdjSalePrice ~  SqFtTotLiving + SqFtFinBasement + SqFtLot + NbrLivingUnits + Bedrooms + BldgGrade + LandVal  + YrRenovated +  ImpsVal + NewConstruction, data = house, na.action=na.omit,weight=wt )

In [ ]:
step = stepAIC(house_model2,direction="both")

# Factors

In [ ]:
#group zip codes into 5 regions ordered by sales price

In [ ]:
library(dplyr)


<code>
zip_groups <- house %>%
mutate(resid = residuals(step)) %>%
  group_by(ZipCode) %>%
  summarize(med_resid = median(resid),
            cnt = n()) %>%
   #sort the zip codes by the median residual
  arrange(med_resid) %>%
  mutate(cum_cnt = cumsum(cnt),
         ZipGroup = factor(ntile(cum_cnt, 5)))
    </code>

In [ ]:
# add zip and residuals to the table, 
by_zip = group_by(mutate(house, resid = residuals(house_model2)),ZipCode)

In [ ]:
# grouping by residuals to better fit previous model
# In other words, values very far off the previous prediction are grouped together, vals close to previos prediction are 
# also together. 
residuals_by_zip = summarize(by_zip,med_resid = median(resid), cnt = n())

In [ ]:
# although already grouped, that doesn't show in the display
residuals_by_zip

In [ ]:
arrange(residuals_by_zip,med_resid)

In [ ]:
residuals_by_zip = residuals_by_zip[order(residuals_by_zip$med_resid),]

In [ ]:
#ntile creates the requested # of fairly equal groups

In [ ]:
ntile(1:11,5)

In [ ]:
# factor creates levels 

In [ ]:
factor(ntile(1:10,5))

In [ ]:
# here we add the the factor column to the zips and their residuals
zip_groups = mutate(residuals_by_zip,cum_cnt = cumsum(cnt=n()), ZipGroup = factor(ntile(cum_cnt, 5)))
zip_groups

In [ ]:
cumsum(1:10)

In [ ]:
# Then we add the new zip_group dataframe to our house dataframe usung the ZipCode

In [ ]:
house = left_join(house,select(zip_groups, ZipCode, ZipGroup), by='ZipCode')
house

In [ ]:
house_model3 = lm(AdjSalePrice ~  SqFtTotLiving + SqFtFinBasement + SqFtLot + NbrLivingUnits + Bedrooms + BldgGrade + LandVal  + YrRenovated +  ImpsVal + NewConstruction + ZipGroup, data = house, na.action=na.omit,weight=wt )

In [ ]:
house_model3

In [ ]:
# And cool .. we lowered our AIC score
# Also predictions for houses in certain zipGroups get adjusted upward to 'fit' better.
step = stepAIC(house_model3,direction="both")

In [ ]:
# the interactions between variables can be observed using the '*' oeprator ... how does ZipGroup affect SqFt?

In [ ]:
house_model4 = lm(AdjSalePrice ~  SqFtTotLiving*ZipGroup + SqFtFinBasement + SqFtLot + NbrLivingUnits + Bedrooms + BldgGrade + LandVal  + YrRenovated +  ImpsVal + NewConstruction , data = house, na.action=na.omit,weight=wt )

In [ ]:
# theres a more dramatic effect on ZipGroup5
house_model4

In [ ]:
step = stepAIC(house_model4,direction="both")

In [ ]:
formula = AdjSalePrice ~ SqFtTotLiving + ZipGroup + SqFtLot + NbrLivingUnits + 
    Bedrooms + BldgGrade + LandVal + YrRenovated + ImpsVal + 
    NewConstruction + SqFtTotLiving:ZipGroup
# save winning formula
formula

## Factor Variables

In [ ]:
#model.matrix breaks a variable into binaries

In [ ]:
props = model.matrix(~PropertyType , data=house)
props

In [ ]:
house_model5 = lm(AdjSalePrice ~  SqFtTotLiving*ZipGroup + SqFtFinBasement + SqFtLot + NbrLivingUnits + Bedrooms + Bathrooms + BldgGrade + LandVal  + YrRenovated +  ImpsVal + NewConstruction + PropertyType , data = house, na.action=na.omit,weight=wt )
house_model5

In [ ]:
hm5 = stepAIC(house_model5,direction="both")
hm5

In [ ]:
hm5$coefficients

## Outliers

In [ ]:
# fit a regression for one ZipCode
house_98105 <- house[house$ZipCode == 98105,]


In [ ]:
zip_model = lm(AdjSalePrice ~  SqFtTotLiving + SqFtFinBasement + SqFtLot + NbrLivingUnits + Bedrooms + Bathrooms + BldgGrade + LandVal  + YrRenovated +  ImpsVal + NewConstruction + PropertyType , data = house_98105, na.action=na.omit,weight=wt )
zip_model

In [ ]:
stepAIC(zip_model,direction="both")

In [ ]:
# get standardized residuals
sr = rstandard(zip_model)

In [ ]:
#smallest residual
ordered = order(sr)
ordered

In [ ]:
sr[25]

In [ ]:
min(sr)

In [ ]:
# which command + min is a nice shortcur for ordering set.
which.min(sr)

In [ ]:
ordered[1]

In [ ]:
nrow(house_98105)

In [ ]:
rstandard(zip_model)[ordered[1]]

In [ ]:
house_98105[23:27,]

In [ ]:
outlier_from_book = house[house$AdjSalePrice == 119748,]


In [ ]:
outlier_from_book

In [ ]:
# turns out the book outlier is next in line with our model:
rstandard(zip_model)[ordered[1:4]]

In [ ]:
### he sorted by residual, not rstandard
ordered2 = order(resid(zip_model))

In [ ]:
ordered2[1]

In [ ]:
which.min(resid(zip_model))

## Testing


In [ ]:
# An INfluence Plot of the 98105 zip records:  Shows hatvalue(x), residuals(y), and cooks distance (point size)
std_resid = rstandard(zip_model)
cooks_D = cooks.distance(zip_model)
hat_values = hatvalues(zip_model)
plot(hat_values, std_resid, cex=3*sqrt(cooks_D))
abline(h=c(-2.5, 2.5), lty=2)

In [ ]:
max(cooks.distance(zip_model))

In [ ]:
tight_model = lm( AdjSalePrice ~  SqFtTotLiving + SqFtFinBasement + SqFtLot + NbrLivingUnits + Bedrooms + Bathrooms + BldgGrade + LandVal  + YrRenovated +  ImpsVal + NewConstruction + PropertyType , subset=cooks.distance(zip_model)<.08, data = house_98105, na.action=na.omit,weight=wt )
tight_model

In [ ]:
steped_up = stepAIC(tight_model,direction="both")

### Partial Residual plot

In [ ]:
library(ggplot2)

terms <- predict(zip_model, type='terms')
partial_resid <- resid(zip_model) + terms

In [ ]:
df = data.frame(SqFtTotLiving = house_98105[, 'SqFtTotLiving'],
                 Terms = terms[, 1],
                 PartialResid = partial_resid[, 1])
ggplot(df, aes(SqFtTotLiving, PartialResid)) +
  geom_point(shape=1) + scale_shape(solid = FALSE) +
  geom_smooth(linetype=2) + 
  geom_line(aes(SqFtTotLiving, Terms))+
  theme_bw()

## Polynomial Regression

In [ ]:
# if relationship to a variable isn't linear, we may square, cube it , etc ...
# In real estate, sqFtLiving isn't linear:

In [ ]:
tighter_model = lm( AdjSalePrice ~  poly(SqFtTotLiving) + SqFtFinBasement + SqFtLot + NbrLivingUnits + Bedrooms + Bathrooms + BldgGrade + LandVal  + YrRenovated +  ImpsVal + NewConstruction + PropertyType , subset=cooks.distance(zip_model)<.08, data = house_98105, na.action=na.omit,weight=wt )
tighter_model

In [ ]:
steped_now = stepAIC(tighter_model,direction="both")
# little difference in predictions

### splines (curves using many lines)

In [ ]:
library(splines)


In [ ]:
knots = quantile(house_98105$SqFtTotLiving, p=c(.25, .5, .75))
#curves line every 3 degrees per knot above
lm_spline = lm(AdjSalePrice ~ bs(SqFtTotLiving, knots=knots, degree=3) +  SqFtLot +  
                  Bathrooms + Bedrooms + BldgGrade,  data=house_98105)
lm_spline

In [ ]:
terms1 = predict(lm_spline, type='terms')
partial_resid1 = resid(lm_spline) + terms

In [ ]:
df1 = data.frame(SqFtTotLiving = house_98105[, 'SqFtTotLiving'],
                 Terms = terms1[, 1],
                 PartialResid = partial_resid1[, 1])


In [ ]:
ggplot(df1, aes(SqFtTotLiving, PartialResid)) +
  geom_point(shape=1) + scale_shape(solid = FALSE) +
  geom_smooth(linetype=2) + 
  geom_line(aes(SqFtTotLiving, Terms))+
  theme_bw()

## Generalized Additive Models

### knots above seem random. library(mgcv)  (gam,s functions) caclulates best knots for us.


In [ ]:
library(mgcv)


In [ ]:
g2_model  = gam( AdjSalePrice ~  s(SqFtTotLiving) + SqFtFinBasement + SqFtLot + NbrLivingUnits + Bedrooms + Bathrooms + BldgGrade + LandVal  + YrRenovated +  ImpsVal + NewConstruction + PropertyType , data = house_98105, na.action=na.omit,weight=wt )

In [ ]:
terms <- predict.gam(g_model, type='terms')
partial_resid <- resid(g_model) + terms

In [ ]:
df = data.frame(SqFtTotLiving = house_98105[, 'SqFtTotLiving'],
                 Terms = terms[, 5],
                 PartialResid = partial_resid[, 5])
ggplot(df, aes(SqFtTotLiving, PartialResid)) +
  geom_point(shape=1) + scale_shape(solid = FALSE) +
  geom_smooth(linetype=2) + 
  geom_line(aes(SqFtTotLiving, Terms))  +
  theme_bw()

In [ ]:
g_model